<a href="https://colab.research.google.com/github/chelsk5/UW-LING-573-Group-5/blob/main/LING573_Train_T5_Ante_Hoc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Authors: Nina Koh, Chelsea Kendrick, Vanesa Marar
# Install dependencies

!pip install -q \
  transformers==4.41.0 \
  datasets==2.19.1 \
  huggingface-hub==0.28.1 \
  peft==0.8.2 \
  evaluate rouge-score nltk

!pip install accelerate==0.27.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 464.1/464.1 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 27.5 MB/s eta 0:00:00
   ━━

In [3]:
# Clone repo & set up environment
!git clone https://github.com/chelsk5/UW-LING-573-Group-5.git
!mv UW-LING-573-Group-5 UW_LING_573_Group_5 # Python modules use underscores

Cloning into 'UW-LING-573-Group-5'...
remote: Enumerating objects: 734, done.
remote: Counting objects: 100% (181/181), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 734 (delta 124), reused 131 (delta 87), pack-reused 553 (from 1)
Receiving objects: 100% (734/734), 601.57 KiB | 4.56 MiB/s, done.
Resolving deltas: 100% (224/224), done.


In [11]:
# %cd UW_LING_573_Group_5
!git checkout main
%ls

Already on 'main'
Your branch is up to date with 'origin/main'.
CreateOpinionGraph.py   LING573_Zero_Shot_T5.ipynb  rouge.py
gold_info.py            mmr.py                      t5-small_results/
lex-rank.py             mmr_results/                tf-idf.py
lexrank_results/        README.md                   tf-idf_results/
LING573_Model.ipynb     requirements.txt
LING573_Train_T5.ipynb  rouge_eval.py


In [28]:
# 1. Load data from local GitHub folder
# import os, re, glob
import json
from datasets import Dataset
from sklearn.model_selection import train_test_split

def clean_line(line):
    line = line.lower()
    # line = re.sub(r'[^\w\s]', '', line)
    line = re.sub(r'\s+', ' ', line)
    return line

def load_from_json(json_path):
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    examples = []
    for topic, reviews in data.items():
        cleaned_reviews = [clean_line(r) for r in reviews]
        full_text = " ".join(cleaned_reviews)
        # Dummy gold summary — replace later if needed
        gold_summary = cleaned_reviews[0] if cleaned_reviews else ""
        examples.append({
            "topic": topic,
            "reviews": full_text,
            "gold_summary": gold_summary
        })
    return examples

# def load_local_opinosis(data_dir):
#     examples = []
#     search_path = os.path.join(data_dir, "*.txt.data")
#     files = glob.glob(search_path)

#     for file in files:
#         topic = os.path.splitext(os.path.basename(file))[0].replace(".txt", "")
#         with open(file, "r", encoding="utf-8", errors="ignore") as f:
#             lines = [clean_line(line.strip()) for line in f if line.strip()]
#             if lines:
#                 full_text = " ".join(lines)
#                 gold_summary = lines[0]  # TEMP: use first line as dummy summary
#                 examples.append({
#                     "topic": topic,
#                     "reviews": full_text,
#                     "gold_summary": gold_summary
#                 })
#     return examples

# Load data and split
# data_dir = "/content/UW_LING_573_Group_5/data"
# all_examples = load_local_opinosis(data_dir)

# train_examples, test_examples = train_test_split(all_examples, test_size=0.2, random_state=1)
# train_data = Dataset.from_list(train_examples)
# test_data = Dataset.from_list(test_examples)

# Load selected reviews from specified JSON file
# json_path = "/content/UW_LING_573_Group_5/tf-idf_results/tfidf_ante-hoc.json"
# json_path = "/content/UW_LING_573_Group_5/lexrank_results/lexrank_ante-hoc.json"
json_path = "/content/UW_LING_573_Group_5/mmr_results/mmr_ante-hoc.json"

examples = load_from_json(json_path)

# Split into train/test and convert to HuggingFace Dataset
train_examples, test_examples = train_test_split(examples, test_size=0.2, random_state=1)
train_data = Dataset.from_list(train_examples)
test_data = Dataset.from_list(test_examples)

In [29]:
# 2. Preprocessing
def clean_data(example):
    """Preserves topic_id, reviews, and gold_summary."""
    return {
        "topic_id": example["topic"], # Keep the 'topic' key and rename it to 'topic_id' for clarity
        "reviews": example["reviews"],
        # We will overwrite gold_summary with the real one later
        "gold_summary": example["gold_summary"] # Keep the dummy summary for now if needed elsewhere, or remove if not
    }

# Apply mapping
cleaned_train = train_data.map(clean_data)
cleaned_test = test_data.map(clean_data)

# Print a couple of examples to check for 'topic_id'
print("Input reviews:")
print(cleaned_train[0]["reviews"])
print("\nTarget summary (dummy):") # Note this is the dummy summary
print(cleaned_train[0]["gold_summary"])
print("\nTopic ID:") # Check that topic_id is now present
print(cleaned_train[0]["topic_id"])


print("\nInput reviews:")
print(cleaned_train[1]["reviews"])
print("\nTarget summary (dummy):") # Note this is the dummy summary
print(cleaned_train[1]["gold_summary"])
print("\nTopic ID:") # Check that topic_id is now present
print(cleaned_train[1]["topic_id"])

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Map:   0%|          | 0/11 [00:00<?, ? examples/s]

Input reviews:
front seats much more comfortable . ride is very good, but seats are just a little firm . front seats are very uncomfortable . i love the seats, they are supportive . leather seats are very comfortable . this accord has more road noise than i like and the seats tend to be hard, unlike my other accords . at first the seats seemed stiffer than i'd like, but after making a 2 1 2 hour trip, it felt very comfortable once i got the seat and lumbar adjusted to my liking .

Target summary (dummy):
front seats much more comfortable .

Topic ID:
seats_honda_accord_2008

Input reviews:
the car is great, both with styling and performance . engine performance is very lacking . good looking inside and out, comfortable and roomy, commendable performance, and excellent gas mileage . very happy with the car enjoy the ride and performance . the car is superb, performance, styling, quality, engineering, fit and finish , the whole banana ! when i first tested the ex, l v6, the performance a

In [30]:
# 3. Load tokenizer & model
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, BitsAndBytesConfig

# quantization_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_compute_dtype="float16",
#     bnb_4bit_quant_type="nf4",  # You can also use "fp4" depending on your needs
#     bnb_4bit_use_double_quant=True
# )

model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_name, quantization_config=quantization_config, device_map="auto")
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [31]:
# 4. Tokenize reviews & gold summaries from Github repo

def load_gold_summary_for_review(topic_id, gold_dir="summaries-gold"):
    """Loads and combines .gold files for a given topic ID from the summaries-gold/ directory."""
    topic_path = os.path.join(gold_dir, topic_id)
    gold_summary = ""
    if os.path.exists(topic_path):
        for fname in os.listdir(topic_path):
            if fname.endswith(".gold"):
                with open(os.path.join(topic_path, fname), "r", encoding="utf-8") as f:
                    gold_summary += f.read().strip() + " "
    return gold_summary.strip()

def tokenize_data(cleaned_data, tokenizer):
    """Convert the raw text into token IDs for model processing using extracted gold summary."""
    topic_id = cleaned_data["topic_id"]
    reviews_text = cleaned_data["reviews"]

    gold_summary = load_gold_summary_for_review(topic_id)

    pref_text = "summarize: " + reviews_text

    tokenized_input = tokenizer(
        pref_text,
        max_length=512,
        truncation=True,
        padding="max_length"
    )

    labels = tokenizer([gold_summary], max_length=64, truncation=True)
    tokenized_input["labels"] = labels["input_ids"][0]

    return tokenized_input

# Example usage (assuming your data includes 'topic_id', 'reviews'):
tokenized_train = [tokenize_data(d, tokenizer) for d in cleaned_train]
tokenized_test = [tokenize_data(d, tokenizer) for d in cleaned_test]

# Check tokenization
print("Tokenized input example (input_ids):")
print(tokenized_train[0]["input_ids"])
print("Tokenized target (labels):")
print(tokenized_train[0]["labels"])


Tokenized input example (input_ids):
[21603, 10, 851, 6116, 231, 72, 1965, 3, 5, 2564, 19, 182, 207, 6, 68, 6116, 33, 131, 3, 9, 385, 1669, 3, 5, 851, 6116, 33, 182, 14209, 3, 5, 3, 23, 333, 8, 6116, 6, 79, 33, 12758, 3, 5, 4295, 6116, 33, 182, 1965, 3, 5, 48, 4408, 65, 72, 1373, 4661, 145, 3, 23, 114, 11, 8, 6116, 2134, 12, 36, 614, 6, 9770, 82, 119, 4408, 7, 3, 5, 44, 166, 8, 6116, 3776, 14537, 49, 145, 3, 23, 31, 26, 114, 6, 68, 227, 492, 3, 9, 204, 209, 204, 1781, 1469, 6, 34, 1800, 182, 1965, 728, 3, 23, 530, 8, 3143, 11, 3, 5171, 1047, 13108, 12, 82, 23340, 3, 5, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [32]:
# 5. Create batch of examples
from transformers import DataCollatorForSeq2Seq

# Dynamically pad sentences to the longest length
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [33]:
# 7. Train model
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
import torch
from accelerate import Accelerator

# Manually tune hyperparameters

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./t5-small-results", # folder to save progress
    evaluation_strategy="epoch", # evaluate after one complete pass
    learning_rate=3e-5,
    per_device_train_batch_size=8, # small batch size for small data
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3, # number of checkpoints (subject to change?)
    num_train_epochs=5, # total passes thru data (also stc...)
    generation_max_length=100,
    predict_with_generate=True,
    fp16=False,# changed to False to avoid ValueError
    report_to="none"
)

# Initialize the trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator)

# Train (finally)
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,7.833783
2,No log,7.075768
3,No log,6.591122
4,No log,6.313440
5,No log,6.205025


TrainOutput(global_step=25, training_loss=7.044909057617187, metrics={'train_runtime': 443.0107, 'train_samples_per_second': 0.451, 'train_steps_per_second': 0.056, 'total_flos': 27068360294400.0, 'train_loss': 7.044909057617187, 'epoch': 5.0})

In [34]:
# 10. *** COPY Save predictions & labels in json files COPY ***
import os, json
from datasets import concatenate_datasets
from torch.utils.data import DataLoader
from transformers import DataCollatorForSeq2Seq
import torch
import numpy as np

# Combine tokenized train and test sets
tokenized_train_ds = Dataset.from_list(tokenized_train)
tokenized_test_ds = Dataset.from_list(tokenized_test)
tokenized_opinosis = concatenate_datasets([tokenized_train_ds, tokenized_test_ds])
print(tokenized_opinosis.column_names)


# Remove raw text fields (adjust names as needed)
try:
    tokenized_opinosis = tokenized_opinosis.remove_columns(
        ["review_sents", "summaries", "reviews", "gold_summary"]
    )
except ValueError as e:
    print(f"Warning: Could not remove columns. Check column names. Error: {e}")
    # You might want to inspect the column names here
    print("Current columns:", tokenized_opinosis.column_names)

# Create DataLoader with automatic padding
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
dataloader = DataLoader(tokenized_opinosis, batch_size=8, collate_fn=data_collator)

# Prepare model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

decoded_preds = []
topic_ids = []

# Extract topic_ids from the original cleaned datasets before they were converted to lists of dictionaries
# or from tokenized_opinosis if topic_id was preserved during tokenization
# Assuming topic_id is not directly in tokenized_opinosis after removing columns,
# we need to get it from the original cleaned data.
# However, the original `cleaned_train` and `cleaned_test` are Datasets, not lists, and the
# `clean_data` function didn't add `topic_id` to the dataset structure itself,
# but rather returned a dictionary with 'topic_id'. The lists `tokenized_train` and
# `tokenized_test` were created from these dictionaries.
# Let's assume the original `all_examples` before the split contains the topic_ids.
# We can map back to the original examples or assume the order is preserved.
# A safer approach is to re-load or re-process to get topic_ids alongside the tokenized data.
# Given the current notebook state, the easiest way is to get topic_ids from the original all_examples
# and assume the order in the combined tokenized dataset matches the order of combining the original lists.

# Re-loading or assuming order might not be robust. Let's re-create a structure that keeps topic_id.
# Instead of removing columns early, let's keep topic_id in the tokenized dataset.
# I'll modify the tokenize_data function to keep topic_id if it exists in the input.
# (This would require modifying the tokenize_data function in cell 5g7E4loULSwI, which I cannot do directly here)

# A workaround for the current state: Since tokenized_train and tokenized_test were created from
# lists of dictionaries which included 'topic_id', we can iterate through the original
# train_examples and test_examples lists to get topic_ids in the correct order.

combined_examples = train_examples + test_examples
for example in combined_examples:
    if "topic" in example: # Check for the original 'topic' key
         topic_ids.append(example["topic"])


# Generate summaries
for batch in dataloader:
    # Ensure batch is moved to the correct device
    batch = {k: v.to(device) for k, v in batch.items() if isinstance(v, torch.Tensor)}
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]

    with torch.no_grad():
        generated_ids = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_length=100,
            num_beams=4,
            early_stopping=True
        )

    decoded_batch = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    decoded_preds.extend([d.strip() for d in decoded_batch])

# Zip topic_ids and decoded_preds together
topic_summary_dict = {}
# Ensure topic_ids and decoded_preds have the same length
if len(topic_ids) == len(decoded_preds):
    for topic, pred in zip(topic_ids, decoded_preds):
        topic_summary_dict[topic] = [pred]
else:
    print(f"Warning: Mismatch between number of topic_ids ({len(topic_ids)}) and generated summaries ({len(decoded_preds)}).")
    # Handle the mismatch, e.g., by only zipping the minimum length
    min_len = min(len(topic_ids), len(decoded_preds))
    for topic, pred in zip(topic_ids[:min_len], decoded_preds[:min_len]):
         topic_summary_dict[topic] = [pred]


# Save generated summaries
with open("mmr_antehoc_summaries.json", "w", encoding="utf-8") as f:
    json.dump(topic_summary_dict, f, indent=2, ensure_ascii=False)

# Decode reference summaries from trainer (just for labels)
predictions = trainer.predict(tokenized_opinosis)
labels = np.where(predictions.label_ids != -100, predictions.label_ids, tokenizer.pad_token_id)
decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
decoded_labels = [label.strip() for label in decoded_labels]

# Save reference summaries
with open("reference_summaries.json", "w", encoding="utf-8") as f:
    json.dump(decoded_labels, f, indent=2, ensure_ascii=False)

# import os, json
# from collections import defaultdict
# from datasets import concatenate_datasets

# # Model-generated summaries
# tokenized_opinosis = concatenate_datasets([tokenized_train, tokenized_test]) # full dataset
# predictions = trainer.predict(tokenized_opinosis, max_length=100) # use full dataset, not just training
# pred_ids = np.argmax(predictions.predictions, axis=-1)
# decoded_preds = tokenizer.batch_decode(predictions.predictions, skip_special_tokens=True)

# with open("t5_summaries.json", "w") as f:
#     json.dump(decoded_preds, f, indent=2)

# # Reference summaries
# labels = np.where(predictions.label_ids != -100, predictions.label_ids, tokenizer.pad_token_id)
# decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

# with open("reference_summaries.json", "w") as f:
#     json.dump(decoded_labels, f, indent=2)

# system_dir = "t5-small-results/system_summaries" # t5 predictions
# model_dir = "t5-small-results/model_summaries" # gold reference
# os.makedirs(system_dir, exist_ok=True)
# os.makedirs(model_dir, exist_ok=True)

# Write to output files, where each file contains a single summary
# for i, (pred, ref) in enumerate(zip(decoded_preds, decoded_labels)):
#     with open(f"{system_dir}/{i}.txt", "w") as sys_f:
#         sys_f.write(pred.strip() + "\n")
#     with open(f"{model_dir}/{i}.A.1.txt", "w") as ref_f:
#         ref_f.write(ref.strip() + "\n")

['input_ids', 'attention_mask', 'labels']
Current columns: ['input_ids', 'attention_mask', 'labels']
